# Baseline

In [20]:
import itertools

from petreader.RelationsExtraction import RelationsExtraction
from petreader.TokenClassification import TokenClassification
from petreader.labels import FLOW, SAME_GATEWAY, AND_GATEWAY, XOR_GATEWAY

In [1]:
relations_dataset = RelationsExtraction()
token_dataset = TokenClassification()

Using the latest cached version of the module from C:\Users\janek\.cache\huggingface\modules\datasets_modules\datasets\patriziobellan--PET\38434e2af57af533c400c8975f37e43c08bb77739085a3c026a862b2efb668d2 (last modified on Sun Jul  3 12:17:36 2022) since it couldn't be found locally at patriziobellan/PET., or remotely on the Hugging Face Hub.
Reusing dataset pet (C:\Users\janek\.cache\huggingface\datasets\patriziobellan___pet\relations-extraction\1.0.1\38434e2af57af533c400c8975f37e43c08bb77739085a3c026a862b2efb668d2)




 _______ _     _ _______       _____  _______ _______      ______  _______ _______ _______ _______ _______ _______
    |    |_____| |______      |_____] |______    |         |     \ |_____|    |    |_____| |______ |______    |   
    |    |     | |______      |       |______    |         |_____/ |     |    |    |     | ______| |______    |   
                                                                                                                  
Discover more at: [https://pdi.fbk.eu/pet-dataset/]



  0%|          | 0/1 [00:00<?, ?it/s]

Using the latest cached version of the module from C:\Users\janek\.cache\huggingface\modules\datasets_modules\datasets\patriziobellan--PET\38434e2af57af533c400c8975f37e43c08bb77739085a3c026a862b2efb668d2 (last modified on Sun Jul  3 12:17:36 2022) since it couldn't be found locally at patriziobellan/PET., or remotely on the Hugging Face Hub.
Reusing dataset pet (C:\Users\janek\.cache\huggingface\datasets\patriziobellan___pet\token-classification\1.0.1\38434e2af57af533c400c8975f37e43c08bb77739085a3c026a862b2efb668d2)




 _______ _     _ _______       _____  _______ _______      ______  _______ _______ _______ _______ _______ _______
    |    |_____| |______      |_____] |______    |         |     \ |_____|    |    |_____| |______ |______    |   
    |    |     | |______      |       |______    |         |_____/ |     |    |    |     | ______| |______    |   
                                                                                                                  
Discover more at: [https://pdi.fbk.eu/pet-dataset/]



  0%|          | 0/1 [00:00<?, ?it/s]

## 1 Prepare Data

### 1.1 Read Example Doc

In [23]:
doc_id = 0
doc_name = token_dataset.GetDocumentName(doc_id)
print(f"  {doc_name}  ".center(50, '*'))
doc_text = relations_dataset.GetDocument(doc_id)
print(doc_text)
doc_activities = token_dataset.GetDocumentActivities(doc_name)

print(" activities and NER labels (per sentences) ".center(50, '*'))
print(doc_activities)
doc_sentence_ner_labels = relations_dataset.GetSentencesWithIdsAndNerTagLabels(doc_id)
print(doc_sentence_ner_labels[:4])
doc_relations = relations_dataset.GetRelations(doc_id)
doc_flow_relations, doc_same_gateway_relations = doc_relations[FLOW], doc_relations[SAME_GATEWAY]

print(" same gateway relations ".center(50, '*'))
for same_gateway_relation in doc_same_gateway_relations:
    for key, value in same_gateway_relation.items():
        print(f"{key}: {value}")
    print()

*******************  doc-1.1  ********************
A small company manufactures customized bicycles . Whenever the sales department receives an order , a new process instance is created . A member of the sales department can then reject or accept the order for a customized bike . In the former case , the process instance is finished . In the latter case , the storehouse and the engineering department are informed . The storehouse immediately processes the part list of the order and checks the required quantity of each part . If the part is available in-house , it is reserved . If it is not available , it is back-ordered . This procedure is repeated for each item on the part list . In the meantime , the engineering department prepares everything for the assembling of the ordered bicycle . If the storehouse has successfully reserved or back-ordered every item of the part list and the preparation activity has finished , the engineering department assembles the bicycle . Afterwards , the s

### 1.2 Preprocess sentences

In [18]:
num_sentences = len(doc_activities) # activities is 2 dim list (one per sentence)
print(num_sentences)
sentences_raw = [sentence.strip() for sentence in doc_text.split(".") if sentence.strip() != ""]
for s in sentences_raw:
    print(s)
assert num_sentences == len(sentences_raw)  # check if number of extracted sentences == from dataset 

12
A small company manufactures customized bicycles
Whenever the sales department receives an order , a new process instance is created
A member of the sales department can then reject or accept the order for a customized bike
In the former case , the process instance is finished
In the latter case , the storehouse and the engineering department are informed
The storehouse immediately processes the part list of the order and checks the required quantity of each part
If the part is available in-house , it is reserved
If it is not available , it is back-ordered
This procedure is repeated for each item on the part list
In the meantime , the engineering department prepares everything for the assembling of the ordered bicycle
If the storehouse has successfully reserved or back-ordered every item of the part list and the preparation activity has finished , the engineering department assembles the bicycle
Afterwards , the sales department ships the bicycle to the customer and finishes the pro

### 1.3 Filter Tokens for Gateways

In [25]:
doc_gateway_tokens = [[token for token in s_list if "Gateway" in token[2]]
                        for s_list in doc_sentence_ner_labels]
doc_gateway_tokens = list(itertools.chain(*doc_gateway_tokens)) # flattened
print(doc_gateway_tokens)

[('or', 9, 'B-XOR Gateway'), ('If', 0, 'B-XOR Gateway'), ('If', 0, 'B-XOR Gateway'), ('In', 0, 'B-AND Gateway'), ('the', 1, 'I-AND Gateway'), ('meantime', 2, 'I-AND Gateway'), ('If', 0, 'B-XOR Gateway')]


### 1.X Filter Flow Relations? TODO
see question doc

### 1.4 Key Word List
#### A) take words from all existing gateways as gold list for detection

In [41]:
def get_gateway_key_words(dataset_gateway_list):
    flattened = list(itertools.chain(*dataset_gateway_list))
    phrases = [" ".join(g).lower() for g in flattened]  # join phrases together if multiple words
    unique = list(set(phrases))
    unique.sort()
    return unique

xor_key_words_gold = get_gateway_key_words(token_dataset.GetXORGateways())
and_key_words_gold = get_gateway_key_words(token_dataset.GetANDGateways())

print(xor_key_words_gold)
print(and_key_words_gold)

['either', 'for', 'for each patient for which', 'for the case', 'if', 'in case', 'in case of', 'in the case of', 'it can also happen that', 'or', 'otherwise', 'should', 'sometimes', 'under certain circumstances', 'whereas']
['at the same time', 'in the meantime', 'meantime', 'two concurrent activities are triggered', 'whereas', 'while']


#### B) Curated List from Literature